In [1]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt

from statistics import mean

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5),(0.5))])

trainset = datasets.MNIST('MNIST_data/', download=True, train=True, transform=transform)
trainloader = T.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset    = datasets.MNIST('MNIST_data/', download=True, train=False, transform=transform)
testloader = T.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [3]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.input_layer = nn.Conv2d(1,28,kernel_size=7)
        self.conv1 = nn.Conv2d(28, 56, 7)
        self.conv2 = nn.Conv2d(56,112,7)
        self.drop = nn.Dropout2d(0.1)
        self.layer1 = nn.Linear(112*10*10,60)
        self.layer2 = nn.Linear(60,30)
        self.layer3 = nn.Linear(30,10)

    def forward(self, x):
        x = F.relu(self.drop(self.input_layer(x)))
        x = F.relu(self.drop(self.conv1(x)))
        x = F.relu(self.drop(self.conv2(x)))
        #print(x.shape)
        x = x.view(x.shape[0], -1)
        #print(x.shape)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        out_layer = self.layer3(x)
        return out_layer


In [4]:
model = Classifier()

In [5]:
model

Classifier(
  (input_layer): Conv2d(1, 28, kernel_size=(7, 7), stride=(1, 1))
  (conv1): Conv2d(28, 56, kernel_size=(7, 7), stride=(1, 1))
  (conv2): Conv2d(56, 112, kernel_size=(7, 7), stride=(1, 1))
  (drop): Dropout2d(p=0.1, inplace=False)
  (layer1): Linear(in_features=11200, out_features=60, bias=True)
  (layer2): Linear(in_features=60, out_features=30, bias=True)
  (layer3): Linear(in_features=30, out_features=10, bias=True)
)

In [6]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [8]:
epochs = 2
print_every = 40
final_trl = []
final_tel =[]
final_accuracy = []


for e in range(epochs):
    running_loss = 0
    train_losses = []
    test_losses = []
    epoch_accurracy =[]
    print(f"Epoch: {e+1}/{epochs}")

    for i, (images, labels) in enumerate(iter(trainloader)):


        # Flatten MNIST images into a 784 long vector
        # images.resize_(images.size()[0], 784)
        T.squeeze(images.resize_(images.size()[0], 784))
        
        optimizer.zero_grad()
        
        logits = model.forward(images)   # 1) Forward pass
        pred = F.log_softmax(logits, dim=1)
        loss = criterion(pred, labels) # 2) Compute loss
        loss.backward()                  # 3) Backward pass
        optimizer.step()                 # 4) Update model
        
        running_loss += loss.item()

        train_losses.append(loss.item())
        
        
        if i % print_every == 0:
            print(f"\tIteration: {i}\t Loss: {running_loss/print_every:.4f}")
            running_loss = 0

    
    model.eval()
    with T.no_grad():
        for i, (images, labels) in enumerate(iter(testloader)):
            images.resize_(images.size()[0], 784)

            logits = model.forward(images)
            test_pred = F.log_softmax(logits, dim=1)

            test_loss = criterion(test_pred, labels)

            test_losses.append(test_loss.item())

            top_p, top_class = test_pred.topk(k=1, dim=1)
            equals = top_class == labels[i].view(*top_class.shape)
            misclassified = [index for index,value in enumerate(equals) if value.item() is False] 
            accuracy = T.mean(equals.type(T.FloatTensor))
            epoch_accurracy.append(accuracy.item()*100)


            



    model.train()
    
    mean_trl = mean(train_losses)
    mean_tel = mean(test_losses)
    mean_acc = mean(epoch_accurracy)
    final_trl.append(mean_trl)
    final_tel.append(mean_tel)
    final_accuracy.append(mean_acc)
    

    #accuracies.append(acc)
    print(f'Epoch: {e + 1} | loss: {loss.item()} | test loss: {test_loss.item()} | accuracy:{accuracy.item()*100} ')

Epoch: 1/2


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [64, 784]

In [ ]:
plt.plot(final_trl, label='train Loss')
plt.plot(final_tel, label='test Loss')
#plt.plot(accuracies, label='accuracy')
plt.legend()
plt.show()